In [ ]:
"""
CS7641 Supervised Learning — Hotel (classification) + US Accidents (regression)
v3.1 — Aligns with Fall 2025 v6 dataset rules; no external encoders; RBF-γ grid; Hotel NN epoch curve; Linear SVM accuracy.

Major points:
 - Encoders: SafeTargetEncoder (smoothed target mean) + SafeFreqEncoder (frequency), sklearn-compatible (no extra deps)
 - Global: StandardScaler, float32 features, int labels, fixed seeds, runtime & peak RAM logging, hardware note
 - CV: StratifiedShuffleSplit (classification) / ShuffleSplit (regression) with fixed n_splits
 - Hotel (~100k): DT, kNN (exact/brute/euclidean), LinearSVM(+calibration), RBF-SVM (γ ∈ {scale, 1/d, 2/d}), NN (SGD-only; ≤15 epochs; param-cap)
 - Accidents (~7M): DT & Linear SVR ≥1M rows; RBF SVR ≤100k; kNN ≤250k train/≤25k test; NN within param-cap
 - Plots: Learning curves (all), Validation (complexity) curves, ROC/PR, calibration & CM (Hotel),
          residuals plot (Accidents), DT importances, SVM diagnostics (margin histogram & SV fraction),
          NN epoch curve (Hotel+Accidents) & width-sweep within param cap
 - Extra credit: NN activation study (Hotel). Skips gracefully if torch not installed.
"""

import os, time, math, platform, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)

FIG_DIR = "sl_outputs/figs"; LOG_DIR = "sl_outputs/logs"
os.makedirs(FIG_DIR, exist_ok=True); os.makedirs(LOG_DIR, exist_ok=True)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE); random.seed(RANDOM_STATE)

# --------- RAM/time helpers ---------
try:
    import psutil
except ImportError:
    psutil = None
import resource

def now_mb():
    try:
        if psutil: return psutil.Process().memory_info().rss / (1024**2)
        return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024.0
    except Exception:
        return np.nan

def hw_note():
    cpu = platform.processor() or "Unknown CPU"
    sys = f"{platform.system()} {platform.release()}"
    print(f"[Hardware] {sys}; CPU: {cpu}; Python {platform.python_version()}")

def time_fit_predict(model, Xtr, ytr, Xte, yte, fit_fn="fit", predict_fn="predict"):
    start_ram = now_mb(); t0 = time.time()
    getattr(model, fit_fn)(Xtr, ytr)
    t1 = time.time()
    yhat = getattr(model, predict_fn)(Xte)
    t2 = time.time()
    peak_ram = max(now_mb(), start_ram)
    return (t1 - t0), (t2 - t1), peak_ram, yhat

# --------- Plot helpers ---------
def savefig(prefix, title):
    fname = f"{prefix}_{title.replace(' ', '_').replace('/', '-')}.png"
    path = os.path.join(FIG_DIR, fname)
    plt.tight_layout(); plt.savefig(path, dpi=160); plt.close()
    print(f"[saved] {path}")

def protocol_card(name, X, y=None, scoring=None, cv=None):
    if y is not None and set(np.unique(y)) <= {0,1}:
        pos = float(np.mean(y))
        print(f"[{name}] n={len(X)} scoring={scoring} cv={cv} prevalence={pos:.3f}")
    else:
        print(f"[{name}] n={len(X)} scoring={scoring} cv={cv}")

# --------- sklearn imports ---------
from sklearn.model_selection import (
    train_test_split, learning_curve, validation_curve,
    StratifiedShuffleSplit, ShuffleSplit, GridSearchCV
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, auc, f1_score, confusion_matrix,
    RocCurveDisplay, PrecisionRecallDisplay, CalibrationDisplay, accuracy_score,
    mean_absolute_error, mean_squared_error, median_absolute_error
)
from sklearn.calibration import CalibratedClassifierCV

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.base import BaseEstimator, TransformerMixin

# --------- OHE helper (version-robust) ---------
def make_ohe():
    try:
        # sklearn ≥1.2
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32)
    except TypeError:
        # older versions
        return OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=np.float32)

# ===============================
# Safe encoders (no ext deps)
# ===============================
class SafeTargetEncoder(BaseEstimator, TransformerMixin):
    """
    Smoothed target mean encoding, sklearn-compatible.
    Handles binary classification (y in {0,1}) and regression (float).
    """
    def __init__(self, cols=None, smoothing=20.0):
        self.cols = cols
        self.smoothing = float(smoothing)
        self.maps_ = {}
        self.global_ = None
        self._cols_in_ = None

    def fit(self, X, y=None):
        if y is None:
            raise ValueError("SafeTargetEncoder requires target y")
        X = pd.DataFrame(X).copy()
        if self.cols is None:
            self._cols_in_ = X.columns.tolist()
        else:
            self._cols_in_ = list(self.cols)
        y = pd.Series(y)
        self.maps_.clear()
        self.global_ = float(np.nanmean(y))
        for c in self._cols_in_:
            vc = X[c].astype("category")
            df = pd.DataFrame({"cat": vc, "y": y})
            grp = df.groupby("cat", observed=True)["y"].agg(["mean","count"])
            m = self.smoothing
            enc = (grp["count"] * grp["mean"] + m * self.global_) / (grp["count"] + m)
            self.maps_[c] = enc.to_dict()
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        out = []
        for c in self._cols_in_:
            mp = self.maps_[c]
            v = X[c].map(mp).astype(np.float32)
            v = v.fillna(np.float32(self.global_))
            out.append(v.values.reshape(-1,1))
        return np.hstack(out).astype(np.float32)

class SafeFreqEncoder(BaseEstimator, TransformerMixin):
    """Normalized frequency (count) encoding, sklearn-compatible."""
    def __init__(self, cols=None):
        self.cols = cols
        self.maps_ = {}
        self._cols_in_ = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()
        if self.cols is None:
            self._cols_in_ = X.columns.tolist()
        else:
            self._cols_in_ = list(self.cols)
        self.maps_.clear()
        for c in self._cols_in_:
            vc = X[c].value_counts(dropna=False, normalize=True)
            self.maps_[c] = vc.to_dict()
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        out = []
        for c in self._cols_in_:
            mp = self.maps_[c]
            v = X[c].map(mp).astype(np.float32)
            v = v.fillna(np.float32(0.0))
            out.append(v.values.reshape(-1,1))
        return np.hstack(out).astype(np.float32)

# --------- Metrics helpers ---------
def pr_auc(y_true, scores):
    p, r, _ = precision_recall_curve(y_true, scores)
    return auc(r, p)

def f1_opt_threshold(y_true, scores):
    p, r, t = precision_recall_curve(y_true, scores)
    f1 = np.where((p+r) > 0, 2*p*r/(p+r), 0)
    j = int(np.argmax(f1))
    thr = t[j-1] if 0 < j < len(t) else 0.5
    return float(thr), float(f1[j])

# --------- Curves ---------
def plot_learning(est, title, X, y, scoring, cv, prefix="Hotel"):
    protocol_card(title, X, y, scoring, cv)
    sizes = np.linspace(0.1, 1.0, 5)
    tr_sizes, tr_s, te_s = learning_curve(
        est, X, y, train_sizes=sizes, cv=cv, scoring=scoring,
        n_jobs=-1, shuffle=True, random_state=RANDOM_STATE
    )
    plt.figure(figsize=(6.2,4.6))
    plt.title(f"{prefix} — Learning Curve: {title}")
    plt.xlabel("Training examples"); plt.ylabel(scoring.upper()); plt.grid(alpha=.3)
    plt.plot(tr_sizes, tr_s.mean(1), 'o-', label="Training")
    plt.plot(tr_sizes, te_s.mean(1), 'o-', label="Cross-val"); plt.legend()
    savefig(prefix, f"LC_{title}")

def plot_validation(est, X, y, pname, prange, scoring, title, cv, prefix="Hotel"):
    protocol_card(title, X, y, scoring, cv)
    tr_s, te_s = validation_curve(est, X, y, param_name=pname, param_range=prange, cv=cv, scoring=scoring, n_jobs=-1)
    plt.figure(figsize=(6.2,4.6))
    plt.title(f"{prefix} — Validation Curve: {title}")
    plt.xlabel(pname); plt.ylabel(scoring.upper()); plt.grid(alpha=.3)
    plt.plot(prange, tr_s.mean(1), 'o-', label="Training")
    plt.plot(prange, te_s.mean(1), 'o-', label="Cross-val"); plt.legend()
    savefig(prefix, f"MC_{title}")

# =========================
# Data loaders & preprocess
# =========================
def load_hotel():
    df = pd.read_csv("hotel_bookings.csv")
    # leakage controls
    drop_leak = [c for c in ['reservation_status','reservation_status_date'] if c in df.columns]
    df = df.drop(columns=drop_leak)
    for c in ['agent','company','children']:
        if c in df.columns:
            df[c] = df[c].fillna(0).astype(int)
    if 'country' in df.columns:
        df['country'] = df['country'].fillna(df['country'].mode()[0])
    y = df['is_canceled'].astype(np.int64)
    X = df.drop(columns=['is_canceled'])
    print(f"[Hotel] rows raw={len(df)}  target=is_canceled")
    return X, y

def hotel_preprocessor(X):
    num = X.select_dtypes(include=np.number).columns.tolist()
    te_cols = [c for c in ['country','agent','company'] if c in X.columns]  # high-card
    cat = [c for c in X.select_dtypes(exclude=np.number).columns if c not in te_cols]
    pre = ColumnTransformer([
        ('num', StandardScaler(), num),
        ('ohe', make_ohe(), cat),
        ('tenc', SafeTargetEncoder(cols=te_cols, smoothing=20.0), te_cols)
    ], remainder='drop')
    to32 = ('to32', FunctionTransformer(lambda Z: Z.astype(np.float32)))
    return Pipeline([('pre', pre), to32])

def load_accidents(path="US_Accidents_March23.csv"):
    df = pd.read_csv(path)
    df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
    df['End_Time']   = pd.to_datetime(df['End_Time'],   errors='coerce')
    df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60.0
    df = df.drop(columns=['Start_Time','End_Time'])
    df = df[df['Duration'] > 0]
    if 'Description' in df.columns: df = df.drop(columns=['Description'])
    for c in df.select_dtypes(include='object').columns: df[c] = df[c].fillna('Unknown')
    for c in df.select_dtypes(include='bool').columns:   df[c] = df[c].astype(int)
    df = df.dropna()
    for c in df.select_dtypes(include=np.number).columns: df[c] = df[c].astype(np.float32)
    y = df['Duration'].astype(np.float32)
    X = df.drop(columns=['Duration','Severity']) if 'Severity' in df.columns else df.drop(columns=['Duration'])
    print(f"[Accidents] rows raw={len(df)}  target=Duration(min)")
    return X, y

def accidents_preprocessor(X, high_card_threshold=20):
    cat_all = X.select_dtypes(exclude=np.number).columns.tolist()
    num = X.select_dtypes(include=np.number).columns.tolist()
    low_card = [c for c in cat_all if X[c].nunique(dropna=False) <= high_card_threshold]
    high_card = [c for c in cat_all if c not in low_card]
    pre = ColumnTransformer([
        ('num', StandardScaler(), num),
        ('ohe', make_ohe(), low_card),
        ('freq', SafeFreqEncoder(cols=high_card), high_card)  # no wide OHE for high-card
    ], remainder='drop')
    to32 = ('to32', FunctionTransformer(lambda Z: Z.astype(np.float32)))
    return Pipeline([('pre', pre), to32])

# =========================
# NN helpers
# =========================
def mlp_params_count(input_dim, hidden, out_dim):
    sizes = [input_dim] + list(hidden) + [out_dim]
    params = 0
    for i in range(len(sizes)-1):
        params += sizes[i]*sizes[i+1] + sizes[i+1]
    return params

def choose_widths(input_dim, target_dim, shallow=True, cap_low=2e5, cap_high=1e6):
    candidates = [128, 256, 512]
    for w in candidates[::-1]:
        hidden = (w, w) if shallow else (w//2, w//2, max(64,w//4), max(64,w//4))
        pcount = mlp_params_count(input_dim, hidden, target_dim)
        if cap_low <= pcount <= cap_high:
            return list(hidden)
    return [128, 128] if shallow else [128, 128, 64, 64]

def nn_width_sweep(prefix, X, y, pre, task="cls", widths=(128,256,512), cv=None):
    scores = []
    for w in widths:
        hidden = (w, w)
        pcount = mlp_params_count(pre.transform(X[:5]).shape[1], hidden, 1 if task=="reg" else 2)
        if not (2e5 <= pcount <= 1e6):
            print(f"[NN width sweep] skip w={w} (params≈{int(pcount):,})")
            continue
        if task=="cls":
            est = Pipeline([('prep', pre), ('post', StandardScaler(with_mean=False)),
                            ('clf', MLPClassifier(hidden_layer_sizes=hidden, solver='sgd', momentum=0.0,
                                                  batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                                                  alpha=1e-4, early_stopping=True, n_iter_no_change=3, max_iter=15,
                                                  random_state=RANDOM_STATE))])
            sc = 'roc_auc'
        else:
            est = Pipeline([('prep', pre), ('post', StandardScaler(with_mean=False)),
                            ('reg', MLPRegressor(hidden_layer_sizes=hidden, solver='sgd', momentum=0.0,
                                                 batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                                                 alpha=1e-4, early_stopping=True, n_iter_no_change=3, max_iter=15,
                                                 random_state=RANDOM_STATE))])
            sc = 'neg_mean_absolute_error'
        tr, te = learning_curve(est, X, y, cv=cv, scoring=sc, train_sizes=np.linspace(0.25,1.0,4),
                                n_jobs=-1, shuffle=True, random_state=RANDOM_STATE)
        scores.append((w, te.mean(axis=1)[-1]))
    if scores:
        ws, s = zip(*scores)
        plt.figure(figsize=(6.2,4.6))
        plt.plot(ws, s, 'o-'); plt.grid(alpha=.3)
        plt.xlabel("width (per layer)"); plt.ylabel(("ROC-AUC" if task=="cls" else "−MAE"))
        plt.title(f"{prefix} — NN Model-Complexity (width sweep; param-cap enforced)")
        savefig(prefix, "NN_Width_Sweep")

# =========================
# HOTEL — Classification
# =========================
def run_hotel():
    print("\n== HOTEL (CLASSIFICATION: is_canceled) ==")
    X, y = load_hotel()
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
    pre = hotel_preprocessor(Xtr)
    cv_cls = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=RANDOM_STATE)
    post = ('post', StandardScaler(with_mean=False))

    dt_clf  = Pipeline([('prep', pre), ('clf', DecisionTreeClassifier(class_weight='balanced', random_state=RANDOM_STATE))])
    knn_clf = Pipeline([('prep', pre), post, ('clf', KNeighborsClassifier(n_neighbors=11, algorithm="brute", metric="euclidean", n_jobs=-1))])
    linSVC  = Pipeline([('prep', pre), post, ('clf', LinearSVC(C=1.0, class_weight='balanced', max_iter=15000, random_state=RANDOM_STATE))])
    rbfSVC  = Pipeline([('prep', pre), post, ('clf', SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=RANDOM_STATE))])

    # Fit pre to inspect input dim for NN sizing & gamma grid
    pre.fit(Xtr, ytr)
    input_dim = pre.transform(Xtr[:5]).shape[1]
    d_hotel = input_dim
    gamma_grid_hotel = ["scale", 1.0/d_hotel, 2.0/d_hotel]

    shallow = choose_widths(input_dim, 2, shallow=True)
    nn_sgd = Pipeline([
        ('prep', pre), post,
        ('clf', MLPClassifier(hidden_layer_sizes=tuple(shallow), solver='sgd',
                              batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                              alpha=1e-4, early_stopping=True, n_iter_no_change=3,
                              max_iter=15, shuffle=True, momentum=0.0, nesterovs_momentum=False,
                              random_state=RANDOM_STATE))
    ])

    # Validation curves
    plot_validation(Pipeline([('prep', pre), ('clf', DecisionTreeClassifier(class_weight='balanced', random_state=RANDOM_STATE))]),
                    Xtr, ytr, "clf__max_depth", [6,10,14,18], "roc_auc", "DT vs depth", cv=cv_cls, prefix="Hotel")
    plot_validation(Pipeline([('prep', pre), post, ('clf', KNeighborsClassifier(algorithm="brute", metric="euclidean"))]),
                    Xtr, ytr, "clf__n_neighbors", [3,5,11,21], "roc_auc", "kNN vs k", cv=cv_cls, prefix="Hotel")
    plot_validation(Pipeline([('prep', pre), post, ('clf', LinearSVC(class_weight='balanced', max_iter=15000, random_state=RANDOM_STATE))]),
                    Xtr, ytr, "clf__C", np.logspace(-2,2,5), "roc_auc", "LinearSVM vs C", cv=cv_cls, prefix="Hotel")
    # Note: we vary C on the curve; γ choices included in GS below.
    plot_validation(Pipeline([('prep', pre), post, ('clf', SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=RANDOM_STATE))]),
                    Xtr, ytr, "clf__C", [0.5,2,8], "roc_auc", "RBF-SVM vs C (γ∈{scale,1/d,2/d})", cv=cv_cls, prefix="Hotel")
    plot_validation(Pipeline([('prep', pre), post, ('clf', MLPClassifier(hidden_layer_sizes=tuple(shallow), solver='sgd',
                                                                        momentum=0.0, batch_size=1024, max_iter=15,
                                                                        random_state=RANDOM_STATE))]),
                    Xtr, ytr, "clf__alpha", [1e-4,5e-4,1e-3], "roc_auc", "NN vs L2(alpha)", cv=cv_cls, prefix="Hotel")

    # Learning curves
    plot_learning(dt_clf,   "Decision Tree",      Xtr, ytr, "roc_auc", cv=cv_cls, prefix="Hotel")
    plot_learning(knn_clf,  "kNN",                Xtr, ytr, "roc_auc", cv=cv_cls, prefix="Hotel")
    plot_learning(linSVC,   "Linear SVM",         Xtr, ytr, "roc_auc", cv=cv_cls, prefix="Hotel")
    plot_learning(rbfSVC,   "RBF SVM",            Xtr, ytr, "roc_auc", cv=cv_cls, prefix="Hotel")
    plot_learning(nn_sgd,   "Neural Net (SGD)",   Xtr, ytr, "roc_auc", cv=cv_cls, prefix="Hotel")

    # Tuning (coarse grids per spec)
    dt_grid  = {"clf__max_depth":[6,10,14,18],
                "clf__min_samples_leaf":[50,100,200],
                "clf__min_samples_split":[100,200,400],
                "clf__max_features":["sqrt","log2",0.5],
                "clf__ccp_alpha":[0.0,1e-4,5e-4,1e-3]}
    dt_tuned  = GridSearchCV(dt_clf,  dt_grid, cv=cv_cls, scoring="roc_auc", n_jobs=-1).fit(Xtr, ytr).best_estimator_
    knn_tuned = GridSearchCV(knn_clf, {"clf__n_neighbors":[5,11,21]}, cv=cv_cls, scoring="roc_auc", n_jobs=-1).fit(Xtr, ytr).best_estimator_
    lin_tuned = GridSearchCV(linSVC,   {"clf__C":[0.1,1,10]}, cv=cv_cls, scoring="roc_auc", n_jobs=-1).fit(Xtr, ytr).best_estimator_
    rbf_tuned = GridSearchCV(
        rbfSVC, {"clf__C":[0.5,2,8], "clf__gamma": gamma_grid_hotel},
        cv=StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=RANDOM_STATE),
        scoring="roc_auc", n_jobs=-1
    ).fit(Xtr, ytr).best_estimator_
    nn_tuned  = GridSearchCV(nn_sgd,   {"clf__alpha":[1e-4,5e-4,1e-3]},
                             cv=StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=RANDOM_STATE),
                             scoring="roc_auc", n_jobs=-1).fit(Xtr, ytr).best_estimator_

    # DT stats & importances
    dt_inner = dt_tuned.named_steps['clf']
    print(f"[DT] depth={dt_inner.get_depth()} leaves={dt_inner.get_n_leaves()} nodes={dt_inner.tree_.node_count}")

    def feature_names_from_pre(prep, X_sample):
        trans = prep.named_steps['pre']
        names = []
        for name, trans_obj, cols in trans.transformers_:
            if name == 'num':
                names += list(cols)
            elif name == 'ohe':
                ohe = trans_obj
                try:
                    names += list(ohe.get_feature_names_out(cols))
                except Exception:
                    # fallback: join with underscores
                    names += [f"{c}_{i}" for c in cols for i in range(len(np.unique(X_sample[c])))]
            else:
                names += list(cols)  # target/freq enc: one col per src col
        return names

    fn = feature_names_from_pre(dt_tuned.named_steps['prep'], Xtr)
    imp = pd.Series(dt_inner.feature_importances_, index=fn).sort_values(ascending=False).head(10)
    plt.figure(figsize=(6.8,4.6))
    imp[::-1].plot(kind='barh'); plt.title("Hotel — DT Top-10 Gini Importances"); plt.grid(alpha=.3, axis='x')
    savefig("Hotel","DT_Top10_Importances")

    # Calibrate LinearSVC
    lin_cal = CalibratedClassifierCV(lin_tuned, method="sigmoid", cv=3).fit(Xtr, ytr)
    models = {
        "DT": dt_tuned, "kNN": knn_tuned,
        "Linear SVM (calibrated)": lin_cal, "RBF SVM": rbf_tuned, "NN (SGD)": nn_tuned
    }

    # Threshold tuning on internal val
    Xs, Xv, ys, yv = train_test_split(Xtr, ytr, test_size=0.2, random_state=RANDOM_STATE, stratify=ytr)
    tuned_thr = {}
    for name, m in models.items():
        s = m.predict_proba(Xv)[:,1] if hasattr(m,"predict_proba") else m.decision_function(Xv)
        thr, _ = f1_opt_threshold(yv, s); tuned_thr[name] = thr

    # ROC/PR
    plt.figure(figsize=(6.6,5)); ax = plt.gca()
    for name, m in models.items():
        s = m.predict_proba(Xte)[:,1] if hasattr(m,"predict_proba") else m.decision_function(Xte)
        RocCurveDisplay.from_predictions(yte, s, name=name, ax=ax)
    ax.set_title("Hotel — ROC curves"); ax.grid(alpha=.3); savefig("Hotel","ROC_All")

    plt.figure(figsize=(6.6,5)); ax = plt.gca()
    for name, m in models.items():
        s = m.predict_proba(Xte)[:,1] if hasattr(m,"predict_proba") else m.decision_function(Xte)
        PrecisionRecallDisplay.from_predictions(yte, s, name=name, ax=ax)
    ax.set_title("Hotel — PR curves"); ax.grid(alpha=.3); savefig("Hotel","PR_All")

    # Calibration (top-2 by PR-AUC)
    scored = []
    for name, m in models.items():
        s = m.predict_proba(Xte)[:,1] if hasattr(m,"predict_proba") else m.decision_function(Xte)
        scored.append((name, pr_auc(yte, s), s))
    top2 = sorted(scored, key=lambda x: x[1], reverse=True)[:2]
    plt.figure(figsize=(6.2,4.6)); ax = plt.gca()
    for name, _, s in top2:
        CalibrationDisplay.from_predictions(yte, s, name=name, ax=ax)
    ax.set_title("Hotel — Calibration (Reliability)"); ax.grid(alpha=.3)
    savefig("Hotel","Calibration_Top2")

    # Confusion matrix at tuned threshold for best model
    best_name, _, best_scores = top2[0]
    thr = tuned_thr[best_name]
    yhat = (best_scores >= thr).astype(int)
    cm = confusion_matrix(yte, yhat)
    plt.figure(figsize=(4.8,4.4))
    plt.imshow(cm, cmap='Blues'); plt.title(f"Hotel — Confusion Matrix @{thr:.3f}\n{best_name}")
    for (i,j),v in np.ndenumerate(cm): plt.text(j, i, int(v), ha='center', va='center')
    plt.xlabel("Predicted"); plt.ylabel("True"); savefig("Hotel","CM_best")

    # SVM diagnostics
    try:
        margins = lin_tuned.decision_function(Xte)
    except Exception:
        margins = models["Linear SVM (calibrated)"].decision_function(Xte)
    plt.figure(figsize=(6.2,4.2))
    plt.hist(margins, bins=40, alpha=0.85)
    plt.title("Hotel — Linear SVM Margin Distribution"); plt.xlabel("margin"); plt.ylabel("count"); plt.grid(alpha=.3)
    savefig("Hotel","LinearSVM_Margin_Hist")

    sv = rbf_tuned.named_steps['clf'].n_support_.sum()
    frac = sv / len(ytr)
    print(f"[RBF-SVM] support vectors: {sv} ({frac:.3f} of train)")

    # Final test metrics (add ACC)
    prev = float(np.mean(yte))
    print(f"\n[Hotel Test] Prevalence (PR-AUC baseline) = {prev:.4f}")
    for name, m in models.items():
        s = m.predict_proba(Xte)[:,1] if hasattr(m,"predict_proba") else m.decision_function(Xte)
        roc = roc_auc_score(yte, s); pr = pr_auc(yte, s)
        thr = tuned_thr[name]
        ybin = (s >= thr).astype(int)
        f1 = f1_score(yte, ybin)
        acc = accuracy_score(yte, ybin)
        print(f"{name:24s} ROC-AUC={roc:.4f} PR-AUC={pr:.4f} F1@{thr:.3f}={f1:.4f} ACC@{thr:.3f}={acc:.4f}")

    # --- Hotel: NN epoch curve (SGD; early-stopping-style loop) ---
    Xtr_es, Xval_es, ytr_es, yval_es = train_test_split(Xtr, ytr, test_size=0.2, random_state=RANDOM_STATE, stratify=ytr)
    nn_es = MLPClassifier(hidden_layer_sizes=tuple(shallow), solver='sgd',
                          batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                          alpha=1e-4, warm_start=True, max_iter=1, shuffle=True,
                          momentum=0.0, nesterovs_momentum=False, random_state=RANDOM_STATE)
    pipe_es = Pipeline([('prep', pre), ('post', StandardScaler(with_mean=False)), ('clf', nn_es)])
    best_pr = -1.0; best_epoch = 0; patience = 3; bad = 0; pr_hist=[]
    for epoch in range(15):
        pipe_es.fit(Xtr_es, ytr_es)
        s = pipe_es.predict_proba(Xval_es)[:,1]
        prv = pr_auc(yval_es, s)
        pr_hist.append(prv)
        if prv > best_pr + 1e-6:
            best_pr = prv; best_epoch = epoch; bad = 0
        else:
            bad += 1
        if bad >= patience: break
    plt.figure(figsize=(6.2,4.6))
    plt.plot(range(1, len(pr_hist)+1), pr_hist, 'o-'); plt.axvline(best_epoch+1, ls='--')
    plt.title(f"Hotel — NN Epoch Curve (best@{best_epoch+1}, PR-AUC={best_pr:.3f})")
    plt.xlabel("epoch"); plt.ylabel("PR-AUC"); plt.grid(alpha=.3)
    savefig("Hotel", "NN_Epoch_Curve")

    # NN width-sweep (within cap)
    nn_width_sweep("Hotel", Xtr, ytr, pre, task="cls", widths=(128,256,512), cv=cv_cls)

    # Runtimes
    runtimes = []
    for name, m in models.items():
        tfit, tpred, peak, _ = time_fit_predict(m, Xtr, ytr, Xte, yte, fit_fn="fit", predict_fn="predict")
        runtimes.append(["Hotel", name, len(Xtr), len(Xte), tfit, tpred, peak])

    return models, (Xtr, Xte, ytr, yte), runtimes, pre, shallow

In [ ]:
hw_note()
hotel_models, hotel_data, hotel_runtime, hotel_pre, hotel_nn_width = run_hotel()

In [ ]:
# =========================
# ACCIDENTS — Regression
# =========================
def run_accidents(path="US_Accidents_March23.csv"):
    print("\n== US ACCIDENTS (REGRESSION: Duration minutes) ==")
    X, y = load_accidents(path)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    pre = accidents_preprocessor(Xtr); pre.fit(Xtr, ytr)
    ntr = len(Xtr); nte = len(Xte)
    print(f"[Accidents] cleaned rows={ntr+nte} → train={ntr} test={nte}")

    # Size rules
    n_linear = min(ntr, 1_500_000)  # target ≥1M if available
    n_rbf   = min(ntr,   100_000)
    n_knn   = min(ntr,   250_000); n_knn_test = min(nte, 25_000)

    sel = np.random.RandomState(RANDOM_STATE).permutation(ntr)
    Xtr_linear, ytr_linear = Xtr.iloc[sel[:n_linear]], ytr.iloc[sel[:n_linear]]
    Xtr_rbf,    ytr_rbf    = Xtr.iloc[sel[:n_rbf]],    ytr.iloc[sel[:n_rbf]]
    Xtr_knn,    ytr_knn    = Xtr.iloc[sel[:n_knn]],    ytr.iloc[sel[:n_knn]]
    Xte_knn,    yte_knn    = Xte.iloc[:n_knn_test],    yte.iloc[:n_knn_test]

    post = ('post', StandardScaler(with_mean=False))
    dt   = Pipeline([('prep', pre), ('reg', DecisionTreeRegressor(random_state=RANDOM_STATE))])
    knn  = Pipeline([('prep', pre), post, ('reg', KNeighborsRegressor(n_neighbors=11, algorithm="brute", metric="euclidean", n_jobs=-1))])
    lsvr = Pipeline([('prep', pre), post, ('reg', LinearSVR(C=1.0, random_state=RANDOM_STATE, max_iter=20000))])
    rsvr = Pipeline([('prep', pre), post, ('reg', SVR(kernel='rbf'))])

    cv_reg = ShuffleSplit(n_splits=3, test_size=0.2, random_state=RANDOM_STATE)

    # Gamma grid for RBF-SVR
    d_acc = pre.transform(Xtr[:5]).shape[1]
    gamma_grid_acc = ["scale", 1.0/d_acc, 2.0/d_acc]

    # Validation curves
    plot_validation(Pipeline([('prep', pre), ('reg', DecisionTreeRegressor(random_state=RANDOM_STATE))]),
                    Xtr_linear, ytr_linear, "reg__max_depth", [6,10,14,18], "neg_mean_absolute_error", "DT vs depth", cv=cv_reg, prefix="Accidents")
    plot_validation(Pipeline([('prep', pre), post, ('reg', KNeighborsRegressor(algorithm="brute", metric="euclidean"))]),
                    Xtr_knn, ytr_knn, "reg__n_neighbors", [3,5,11,21], "neg_mean_absolute_error", "kNN vs k", cv=cv_reg, prefix="Accidents")
    plot_validation(Pipeline([('prep', pre), post, ('reg', LinearSVR(max_iter=20000, random_state=RANDOM_STATE))]),
                    Xtr_linear, ytr_linear, "reg__C", np.logspace(-2,2,5), "neg_mean_absolute_error", "LinearSVR vs C", cv=cv_reg, prefix="Accidents")
    # Note: we vary C on the curve; γ choices will be used in GS.
    plot_validation(Pipeline([('prep', pre), post, ('reg', SVR(kernel='rbf'))]),
                    Xtr_rbf, ytr_rbf, "reg__C", [1,10], "neg_mean_absolute_error", "RBF SVR vs C (γ∈{scale,1/d,2/d})", cv=cv_reg, prefix="Accidents")
    plot_validation(Pipeline([('prep', pre), post, ('reg', MLPRegressor(solver='sgd', momentum=0.0, batch_size=1024, max_iter=15, random_state=RANDOM_STATE))]),
                    Xtr_linear, ytr_linear, "reg__alpha", [1e-4,5e-4,1e-3], "neg_mean_absolute_error", "NN vs L2(alpha)", cv=cv_reg, prefix="Accidents")

    # Learning curves
    def LC(est, title, Xlc, ylc): plot_learning(est, title, Xlc, ylc, "neg_mean_absolute_error", cv=cv_reg, prefix="Accidents")
    LC(dt,   "Decision Tree",      Xtr_linear, ytr_linear)
    LC(knn,  "kNN",                Xtr_knn,    ytr_knn)
    LC(lsvr, "Linear SVR",         Xtr_linear, ytr_linear)
    LC(rsvr, "RBF SVR",            Xtr_rbf,    ytr_rbf)
    # NN sizing
    input_dim = pre.transform(Xtr[:5]).shape[1]
    shallow = choose_widths(input_dim, 1, shallow=True)
    nn  = Pipeline([('prep', pre), post,
                    ('reg', MLPRegressor(hidden_layer_sizes=tuple(shallow), solver='sgd',
                                         batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                                         alpha=1e-4, early_stopping=True, n_iter_no_change=3,
                                         max_iter=15, shuffle=True, momentum=0.0,
                                         random_state=RANDOM_STATE))])
    LC(nn,   "Neural Net (SGD)",   Xtr_linear, ytr_linear)

    # Tuning
    dt_grid  = {"reg__max_depth":[6,10,14,18],
                "reg__min_samples_leaf":[100,200],
                "reg__min_samples_split":[200,400],
                "reg__max_features":["sqrt","log2",0.5],
                "reg__ccp_alpha":[0.0,1e-4,5e-4,1e-3]}
    dt_g  = GridSearchCV(dt,  dt_grid, cv=cv_reg, scoring="neg_mean_absolute_error", n_jobs=-1).fit(Xtr_linear, ytr_linear).best_estimator_
    knn_g = GridSearchCV(knn, {"reg__n_neighbors":[5,11,21]}, cv=cv_reg, scoring="neg_mean_absolute_error", n_jobs=-1).fit(Xtr_knn, ytr_knn).best_estimator_
    lsvr_g= GridSearchCV(lsvr,{"reg__C":[0.1,1,10]}, cv=cv_reg, scoring="neg_mean_absolute_error", n_jobs=-1).fit(Xtr_linear, ytr_linear).best_estimator_
    rsvr_g= GridSearchCV(
        rsvr, {"reg__C":[1,10], "reg__gamma": gamma_grid_acc},
        cv=cv_reg, scoring="neg_mean_absolute_error", n_jobs=-1
    ).fit(Xtr_rbf, ytr_rbf).best_estimator_
    nn_g  = GridSearchCV(nn,  {"reg__alpha":[1e-4,5e-4,1e-3]}, cv=cv_reg, scoring="neg_mean_absolute_error", n_jobs=-1).fit(Xtr_linear, ytr_linear).best_estimator_

    models = {"DT": dt_g, "kNN": knn_g, "Linear SVR": lsvr_g, "RBF SVR": rsvr_g, "NN (SGD)": nn_g}

    # Test metrics & residuals
    print("\n[Accidents Test] MAE / MedAE / RMSE / predict-time (s)")
    runtimes = []
    for name, m in models.items():
        Xtr_fit = Xtr_linear if name!="RBF SVR" else Xtr_rbf
        ytr_fit = ytr_linear if name!="RBF SVR" else ytr_rbf
        Xte_eval= Xte if name!="kNN" else Xte_knn
        yte_eval= yte if name!="kNN" else yte_knn
        tfit, tpred, peak, pred = time_fit_predict(m, Xtr_fit, ytr_fit, Xte_eval, yte_eval)
        mae = mean_absolute_error(yte_eval, pred)
        med = median_absolute_error(yte_eval, pred)
        rmse= math.sqrt(mean_squared_error(yte_eval, pred))
        print(f"{name:12s} MAE={mae:.2f} MedAE={med:.2f} RMSE={rmse:.2f}  fit={tfit:.1f}s pred={tpred:.2f}s RAM~{peak:.0f}MB")
        runtimes.append(["Accidents", name, len(Xtr_fit), len(Xte_eval), tfit, tpred, peak])

    # Residuals for best MAE
    maes = {name: mean_absolute_error(yte if name!="kNN" else yte_knn,
                                      models[name].predict(Xte if name!="kNN" else Xte_knn))
            for name in models}
    best = min(maes, key=maes.get)
    ypred = models[best].predict(Xte)
    resid = (yte - ypred)
    plt.figure(figsize=(6.2,4.6))
    plt.scatter(ypred, resid, s=6, alpha=.35)
    plt.axhline(0, ls='--'); plt.xlabel("Predicted duration (min)"); plt.ylabel("Residual (true - pred)")
    plt.title(f"Accidents — Residuals vs Prediction ({best})")
    savefig("Accidents","Residuals_vs_Pred")

    # NN width-sweep (within cap)
    nn_width_sweep("Accidents", Xtr_linear, ytr_linear, pre, task="reg", widths=(128,256,512), cv=cv_reg)

    # NN epoch curve (early-stopping marker)
    Xtr_es, Xval_es, ytr_es, yval_es = train_test_split(Xtr_linear, ytr_linear, test_size=0.2, random_state=RANDOM_STATE)
    nn_es = MLPRegressor(hidden_layer_sizes=tuple(shallow), solver='sgd',
                         batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                         alpha=1e-4, warm_start=True, max_iter=1, shuffle=True,
                         momentum=0.0, random_state=RANDOM_STATE)
    pipe_es = Pipeline([('prep', pre), ('post', StandardScaler(with_mean=False)), ('reg', nn_es)])
    best_mae = np.inf; best_epoch = 0; patience = 3; bad = 0; hist=[]
    for epoch in range(15):
        pipe_es.fit(Xtr_es, ytr_es)
        pred = pipe_es.predict(Xval_es)
        mae = mean_absolute_error(yval_es, pred)
        hist.append(mae)
        if mae < best_mae - 1e-6:
            best_mae = mae; best_epoch = epoch; bad = 0
        else:
            bad += 1
        if bad >= patience: break
    plt.figure(figsize=(6.2,4.6))
    plt.plot(range(1, len(hist)+1), hist, 'o-'); plt.axvline(best_epoch+1, ls='--')
    plt.title(f"Accidents — NN Epoch Curve (best@{best_epoch+1}, MAE={best_mae:.2f})"); plt.xlabel("epoch"); plt.ylabel("MAE"); plt.grid(alpha=.3)
    savefig("Accidents", "NN_Epoch_Curve")

    # DT stats & importances
    dt_inner = dt_g.named_steps['reg']
    print(f"[DT-Reg] depth={dt_inner.get_depth()} leaves={dt_inner.get_n_leaves()} nodes={dt_inner.tree_.node_count}")
    # names for Accidents
    trans = dt_g.named_steps['prep'].named_steps['pre']
    names = []
    for nm, tr, cols in trans.transformers_:
        if nm == 'num':
            names += list(cols)
        elif nm == 'ohe':
            ohe = tr
            try:
                names += list(ohe.get_feature_names_out(cols))
            except Exception:
                names += [f"{c}_{i}" for c in cols for i in range(2)]
        else:
            names += list(cols)
    imp = pd.Series(dt_inner.feature_importances_, index=names).sort_values(ascending=False).head(10)
    plt.figure(figsize=(6.8,4.6))
    imp[::-1].plot(kind='barh'); plt.title("Accidents — DT Top-10 Importances"); plt.grid(alpha=.3, axis='x')
    savefig("Accidents","DT_Top10_Importances")

    return models, (Xtr, Xte, ytr, yte), runtimes, pre

acc_models, acc_data, acc_runtime, acc_pre = run_accidents("US_Accidents_March23.csv")

In [ ]:
cols = ["dataset","model","n_train","n_test","fit_sec","pred_sec","peak_ram_mb"]
rt = pd.DataFrame(hotel_runtime + acc_runtime, columns=cols)
rt.to_csv(os.path.join(LOG_DIR, "runtime_table.csv"), index=False)
print(f"[saved] {os.path.join(LOG_DIR,'runtime_table.csv')}")

In [ ]:
# =========================
# Extra Credit: Activation study (Hotel)
# =========================
def activation_study_hotel(pre_fitted, Xtr, Xte, ytr, yte, width_tuple):
    try:
        import torch
        import torch.nn as nn
        from torch.utils.data import TensorDataset, DataLoader
    except Exception as e:
        print(f"[ExtraCredit] PyTorch not available ({e}); skipping GELU/SiLU. Running ReLU/tanh fallback.")
        results = []
        for act in ["relu", "tanh"]:
            clf = MLPClassifier(hidden_layer_sizes=width_tuple, solver='sgd', momentum=0.0, nesterovs_momentum=False,
                                batch_size=1024, learning_rate='constant', learning_rate_init=0.05,
                                alpha=1e-4, max_iter=15, early_stopping=True, n_iter_no_change=3,
                                random_state=RANDOM_STATE, activation=act)
            pipe = Pipeline([('prep', pre_fitted), ('post', StandardScaler(with_mean=False)), ('clf', clf)])
            pipe.fit(Xtr, ytr)
            s = pipe.predict_proba(Xte)[:,1]
            roc = roc_auc_score(yte, s); pr = pr_auc(yte, s)
            results.append([act, roc, pr])
        pd.DataFrame(results, columns=["activation","ROC","PR"]).to_csv(os.path.join(LOG_DIR,"extra_credit_activation_summary.csv"), index=False)
        return

    torch.manual_seed(RANDOM_STATE); np.random.seed(RANDOM_STATE)
    Xtr_t = pre_fitted.transform(Xtr).astype(np.float32)
    Xte_t = pre_fitted.transform(Xte).astype(np.float32)
    d = Xtr_t.shape[1]

    def make_loader(X, y, bs=1024, shuffle=True):
        from torch.utils.data import TensorDataset, DataLoader
        ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y.values.astype(np.int64)))
        return DataLoader(ds, batch_size=bs, shuffle=shuffle)

    train_loader = make_loader(Xtr_t, ytr)
    val_loader   = make_loader(Xte_t, yte, shuffle=False)

    class MLP(nn.Module):
        def __init__(self, act_name):
            super().__init__()
            acts = {"relu": nn.ReLU(), "gelu": nn.GELU(), "silu": nn.SiLU(), "tanh": nn.Tanh()}
            self.net = nn.Sequential(
                nn.Linear(d, width_tuple[0]), acts[act_name],
                nn.Linear(width_tuple[0], width_tuple[1]), acts[act_name],
                nn.Linear(width_tuple[1], 1)
            )
        def forward(self, x): return self.net(x)

    activations = ["relu", "gelu", "silu", "tanh"]
    results = []
    bce = torch.nn.BCEWithLogitsLoss()
    for act in activations:
        model = MLP(act)
        opt = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.0, weight_decay=1e-4)
        best_val = -1; best_epoch = 0; patience=3; bad=0
        pr_hist=[]; roc_hist=[]
        for epoch in range(15):
            model.train()
            for xb, yb in train_loader:
                opt.zero_grad(); logits = model(xb).squeeze(1)
                loss = bce(logits, yb.float()); loss.backward(); opt.step()
            model.eval()
            with torch.no_grad():
                logits = []
                for xb, _ in val_loader:
                    logits.append(model(xb).squeeze(1))
                logits = torch.cat(logits).detach().numpy()
                roc = roc_auc_score(yte, logits); pr = pr_auc(yte, logits)
                roc_hist.append(roc); pr_hist.append(pr)
            if pr > best_val: best_val = pr; best_epoch = epoch; bad=0
            else: bad += 1
            if bad >= patience: break
        plt.figure(figsize=(6.2,4.6))
        plt.plot(range(1, len(pr_hist)+1), pr_hist, 'o-'); plt.grid(alpha=.3)
        plt.title(f"Hotel — NN Activation ({act.upper()}) PR-AUC vs epoch (best@{best_epoch+1})")
        plt.xlabel("Epoch"); plt.ylabel("PR-AUC"); savefig("Hotel", f"NN_Activation_{act.upper()}_EpochCurve")
        results.append([act, pr_hist[-1], roc_hist[-1], best_epoch+1])
    pd.DataFrame(results, columns=["activation","PR_AUC","ROC_AUC","best_epoch"]).to_csv(
        os.path.join(LOG_DIR,"extra_credit_activation_summary.csv"), index=False)
    print("[ExtraCredit] Wrote sl_outputs/logs/extra_credit_activation_summary.csv")

# Extra credit (optional; will skip if torch not installed)
activation_study_hotel(hotel_pre, hotel_data[0], hotel_data[1], hotel_data[2], hotel_data[3], tuple(hotel_nn_width))